In [1]:
import scipy.optimize as sco

import typing
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math as m
import scipy as sp
import pandas_datareader as pd_data

### Need to define inputs (type: pd.Dataframe):
1. expRtns: "LIBOR rates & forecasted returns"
2. cov: "covariance based on historical data"
3. mu_f: "US treasury yield"

In [2]:
def Portfolio_stats(weights: "porportions of capital", 
                    expRtns: "LIBOR rates & forecasted returns", 
                    cov: "covariance based on historical data", 
                    mu_f: "US treasury yield") -> "portfolio returns,variance,volatility,sharpe_ratio":
    varP = np.dot(weights.T, np.dot(cov,weights))
    volP = np.sqrt(varP)
    rtnP = np.sum(weights*expRtns)
    sharpeP = (rtnP-mu_f)/volP
    return rtnP, varP, volP, sharpeP

In [3]:
def negative_sharpe(weights):
    return -Portfolio_stats(weights,expRtns,cov,mu_f)[3]

In [4]:
def returns_from_prices(prices, log_returns=False):
    """
    Calculate the returns given prices.
    :param prices: adjusted (daily) closing prices of the asset, each row is a
                   date and each column is a ticker/id.
    :type prices: pd.DataFrame
    :param log_returns: whether to compute using log returns
    :type log_returns: bool, defaults to False
    :return: (daily) returns
    :rtype: pd.DataFrame
    """
    if log_returns:
        return np.log(prices).diff().dropna(how="all")
    else:
        return prices.pct_change().dropna(how="all")
        
def _pair_exp_cov(X, Y, span=180):
    """
    Calculate the exponential covariance between two timeseries of returns.
    :param X: first time series of returns
    :type X: pd.Series
    :param Y: second time series of returns
    :type Y: pd.Series
    :param span: the span of the exponential weighting function, defaults to 180
    :type span: int, optional
    :return: the exponential covariance between X and Y
    :rtype: float
    """
    covariation = (X - X.mean()) * (Y - Y.mean())
    # Exponentially weight the covariation and take the mean
    if span < 10:
        warnings.warn("it is recommended to use a higher span, e.g 30 days")
    return covariation.ewm(span=span).mean().iloc[-1]     

def exp_cov(prices, returns_data=False, span=180, frequency=252, **kwargs):
    """
    Estimate the exponentially-weighted covariance matrix, which gives
    greater weight to more recent data.
    :param prices: adjusted closing prices of the asset, each row is a date
                   and each column is a ticker/id.
    :type prices: pd.DataFrame
    :param returns_data: if true, the first argument is returns instead of prices.
    :type returns_data: bool, defaults to False.
    :param span: the span of the exponential weighting function, defaults to 180
    :type span: int, optional
    :param frequency: number of time periods in a year, defaults to 252 (the number
                      of trading days in a year)
    :type frequency: int, optional
    :return: annualised estimate of exponential covariance matrix
    :rtype: pd.DataFrame
    """
    if not isinstance(prices, pd.DataFrame):
        warnings.warn("data is not in a dataframe", RuntimeWarning)
        prices = pd.DataFrame(prices)
    assets = prices.columns
    if returns_data:
        returns = prices
    else:
        returns = returns_from_prices(prices)
    N = len(assets)

    # Loop over matrix, filling entries with the pairwise exp cov
    S = np.zeros((N, N))
    for i in range(N):
        for j in range(i, N):
            S[i, j] = S[j, i] = _pair_exp_cov(
                returns.iloc[:, i], returns.iloc[:, j], span
            )
    cov = pd.DataFrame(S * frequency, columns=assets, index=assets)
    
    return cov


In [5]:
start_date = '2015-09-01'
end_date = '2016-09-01' # hypothetically the current time
ahead_date = '2017-09-01'

symbols = ['AAPL', 'GS', 'GC=F', 'GE']

# historical data: to be used to compute variance
df = pd_data.DataReader(symbols,'yahoo',start_date,end_date)['Adj Close'] 
df = df.dropna()

# one-year-ahead data: to be used as the (fake) forecasted return
df2 = pd_data.DataReader(symbols,'yahoo',end_date,ahead_date)['Adj Close'] 
df2 = df2.dropna()

In [6]:
# import LIBOR rates data
LIBOR_rates = pd.read_csv("LIBOR_USD.csv")
LIBOR_rates

,Date,Week day,ON,1W,1M,2M,3M,6M,12M
0,12.02.2021,Fri,0.07963,0.09125,0.10738,0.15075,0.19375,0.20075,0.29975
1,11.02.2021,Thu,0.07975,0.08663,0.11225,0.15188,0.19763,0.20838,0.30363
2,10.02.2021,Wed,0.07988,0.09050,0.10950,0.15363,0.20088,0.20800,0.30513
3,09.02.2021,Tue,0.07988,0.08538,0.11588,0.15650,0.20250,0.20800,0.30563
4,08.02.2021,Mon,0.08125,0.08163,0.12050,0.15588,0.19538,0.20750,0.30638
...,...,...,...,...,...,...,...,...,...
5079,08.01.2001,Mon,6.01500,6.03750,5.88375,5.75000,5.61625,5.41938,5.30250
5080,05.01.2001,Fri,6.01625,6.05250,5.93625,5.80500,5.69500,5.51625,5.36500
5081,04.01.2001,Thu,6.09625,6.08125,6.05000,5.95375,5.86625,5.70750,5.55500
5082,03.01.2001,Wed,6.65375,6.57875,6.50750,6.39375,6.28625,6.05375,5.78875


In [7]:
# convert to datetime
LIBOR_rates['Date'] = pd.to_datetime(LIBOR_rates['Date'],format="%d.%m.%Y")

In [8]:
# LIBOR rates between start date & end date
Rates = LIBOR_rates.loc[ (LIBOR_rates['Date'] >= start_date) & (LIBOR_rates['Date'] <= end_date) ]
Rates.sort_values(by='Date',ascending=True,inplace=True)
Rates.set_index('Date', inplace=True)
Rates

C:\Users\YH\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Week day,ON,1W,1M,2M,3M,6M,12M
Date,,,,,,,,
2015-09-01,Tue,0.13200,0.15400,0.20120,0.27200,0.33400,0.54275,0.85555
2015-09-02,Wed,0.13300,0.15550,0.20280,0.27100,0.33250,0.53925,0.85430
2015-09-03,Thu,0.13550,0.15450,0.20430,0.27200,0.33350,0.53900,0.85355
2015-09-04,Fri,0.13550,0.15350,0.19925,0.26825,0.33200,0.53750,0.85030
2015-09-07,Mon,0.13550,0.15430,0.20270,0.26900,0.33300,0.54175,0.86005
...,...,...,...,...,...,...,...,...
2016-08-25,Thu,0.41989,0.44217,0.52383,0.65122,0.82933,1.22428,1.52611
2016-08-26,Fri,0.41833,0.44244,0.52439,0.65367,0.83344,1.23150,1.53656
2016-08-30,Tue,0.41944,0.44383,0.52322,0.66356,0.84211,1.24450,1.55933


### LIBOR rates are quoted as <ins>annual</ins> interest rates (in %).
##### To get <ins>daily</ins> percentage change, need to divide by 252 (#trading days in a year).

## Compute covariance (historical data)

In [9]:
df_all = df.copy()
df_all['Cash'] = Rates['ON']/100/252 # short-term interest rates (OverNight LIBOR)
df_all = df_all.dropna()
dfrtn_all = df_all.pct_change().dropna() # the percentage (not log) returns of stocks

In [10]:
cov = exp_cov(dfrtn_all, returns_data = True) #covariance of percentage returns
cov

Symbols,AAPL,GS,GC=F,GE,Cash
Symbols,,,,,
AAPL,0.053325,0.026696,-0.005754,0.014783,-0.012976
GS,0.026696,0.065308,-0.016674,0.026396,-0.013795
GC=F,-0.005754,-0.016674,0.023300,-0.006589,-0.011478
GE,0.014783,0.026396,-0.006589,0.026866,-0.004708
Cash,-0.012976,-0.013795,-0.011478,-0.004708,0.934047


## Compute expected returns (one-year-ahead data)

In [11]:
# expected returns: current short-term rate for cash, and (heuristically) the one-year-ahead average percentage returns for stocks
expRtns = df2.pct_change().mean(axis=0)
expRtns['Cash'] = Rates['ON'][end_date]/100/252
expRtns

Symbols
AAPL    0.001862
GS      0.001323
GC=F    0.000066
GE     -0.000695
Cash    0.000017
dtype: float64

## Use LIBOR rate for short-term risk-free rate
### => for daily expected return of cash
##### Tenor of rate = rebalancing period

## Use Treasury yield for long-term risk-free rate
### => for performance measure, Sharpe ratio, etc.
##### Tenor of rate = portfolio holding period

In [12]:
TreasuryYields = pd.read_csv('USTREASURY_YIELD.csv')
TreasuryYields

,Date,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
0,2021-03-01,0.03,0.03,0.05,0.07,0.08,0.13,0.27,0.71,1.12,1.45,2.11,2.23
1,2021-02-26,0.04,0.04,0.04,0.05,0.08,0.14,0.30,0.75,1.15,1.44,2.08,2.17
2,2021-02-25,0.04,0.03,0.04,0.06,0.09,0.17,0.34,0.81,1.23,1.54,2.25,2.33
3,2021-02-24,0.03,0.03,0.03,0.05,0.08,0.12,0.24,0.62,1.02,1.38,2.07,2.24
4,2021-02-23,0.03,0.02,0.04,0.05,0.08,0.11,0.22,0.59,1.00,1.37,2.03,2.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7790,1990-01-08,NaN,NaN,7.79,7.88,7.81,7.90,7.95,7.92,8.05,8.02,NaN,8.09
7791,1990-01-05,NaN,NaN,7.79,7.85,7.79,7.90,7.94,7.92,8.03,7.99,NaN,8.06
7792,1990-01-04,NaN,NaN,7.84,7.90,7.82,7.92,7.93,7.91,8.02,7.98,NaN,8.04
7793,1990-01-03,NaN,NaN,7.89,7.94,7.85,7.94,7.96,7.92,8.04,7.99,NaN,8.04


In [13]:
TreasuryYields['Date']= pd.to_datetime(TreasuryYields['Date'],format="%Y-%m-%d")
TreasuryYields

,Date,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
0,2021-03-01,0.03,0.03,0.05,0.07,0.08,0.13,0.27,0.71,1.12,1.45,2.11,2.23
1,2021-02-26,0.04,0.04,0.04,0.05,0.08,0.14,0.30,0.75,1.15,1.44,2.08,2.17
2,2021-02-25,0.04,0.03,0.04,0.06,0.09,0.17,0.34,0.81,1.23,1.54,2.25,2.33
3,2021-02-24,0.03,0.03,0.03,0.05,0.08,0.12,0.24,0.62,1.02,1.38,2.07,2.24
4,2021-02-23,0.03,0.02,0.04,0.05,0.08,0.11,0.22,0.59,1.00,1.37,2.03,2.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7790,1990-01-08,NaN,NaN,7.79,7.88,7.81,7.90,7.95,7.92,8.05,8.02,NaN,8.09
7791,1990-01-05,NaN,NaN,7.79,7.85,7.79,7.90,7.94,7.92,8.03,7.99,NaN,8.06
7792,1990-01-04,NaN,NaN,7.84,7.90,7.82,7.92,7.93,7.91,8.02,7.98,NaN,8.04
7793,1990-01-03,NaN,NaN,7.89,7.94,7.85,7.94,7.96,7.92,8.04,7.99,NaN,8.04


In [14]:
# Treasury yield at current time
# assuming portfolio holding period = 3 years
mu_f = TreasuryYields.loc[TreasuryYields['Date']==end_date]['3 YR'].values[0]/100/252
mu_f

3.6111111111111116e-05

## Compute optimal weights

In [15]:
# total number of stocks (+cash)
nn = len(expRtns)

# Initialise weights
w0 = [1.0/nn for i in range(nn)]
w0 = np.array(w0)

# Constraints on weights
cons = ({'type':'eq','fun': lambda x: np.sum(x)-1}) #add up to 1
bnds = tuple((0,1) for x in range(nn)) #only between 0 and 1, i.e. no short-selling

# Maximise sharpes ratio
opts = sco.minimize(negative_sharpe, w0 , method = 'SLSQP', bounds= bnds, constraints = cons)

# Optimal weights
w_opt = opts['x'].round(3)
w_opt

array([0.489, 0.199, 0.298, 0.   , 0.014])

# Things to do:
1. #units of stocks (compute based on optimal weights)
2. Performance on portfolio (plots, P&L, max. draw-down, turn-over rate, risk measures, compare with S&P index)